In [23]:
from FR3Py.sim.mujoco import FR3Sim

In [24]:
robot = FR3Sim(interface_type='torque')

In [10]:
import numpy as np
import time
for i in range(100000):
    robot.step()
    robot.setCommands(np.array([0.1, 0, 0, 0, 0, 0, 0]))
    time.sleep(robot.dt)

KeyboardInterrupt: 

In [21]:
robot.getJointStates()

{'q': array([-5.18364912e-06, -7.85505698e-01,  3.49774282e-06, -2.35652277e+00,
        -8.37503678e-06,  1.57086361e+00,  7.85395454e-01]),
 'dq': array([-0.00172577, -0.03581088,  0.00116629, -0.10935601, -0.00279364,
         0.02233195, -0.00090079]),
 'tau_est': array([ 8.62351419e-04,  4.00571200e+00,  6.43451587e-01, -2.19662766e+01,
        -6.32432804e-01, -2.28925779e+00,  4.51446180e-04])}

In [33]:
import time
import numpy as np
from FR3Py.controllers.jacobianPseudoInv import WaypointController

controller = WaypointController(kp=1.5)
# Read the initial state of the robot
time.sleep(0.2)
robot.reset()
state = robot.getJointStates()
for i, f in enumerate(controller.robot.robot.model.frames):
    print(f"{i}: {f.name}")
if state is not None:
    print('starting the controller')
    q, dq = state['q'], state['dq']
    q = np.hstack([q, np.zeros(2)])
    dq = np.hstack([dq, np.zeros(2)]
                   )
    p0 = controller.robot.getInfo(q,dq)['P_EE']
    R0 = controller.robot.getInfo(q,dq)['R_EE']
    T0 = np.vstack([np.hstack([R0, p0.reshape(3,1)]), np.array([0,0,0,1])])
    
    start_time = time.time()
    while time.time()-start_time < 10:
        time.sleep(robot.dt)
        t = time.time() - start_time
        x = 0.1 * np.sin(t)
        T= T0 @ np.vstack([np.hstack([np.eye(3), np.array([0,0,x]).reshape(3,1)]), np.array([0,0,0,1])])
        state = robot.getJointStates()
        q, dq = state['q'], state['dq']
        cmd = controller.compute(q, dq, T_cmd=T)
        robot.setCommands(cmd[:7])
        robot.step()

    robot.setCommands(np.zeros(7))
    print('Demo ended.')
else:
    print('State is None. check the connection')

0: universe
1: fr3_link0
2: fr3_joint1
3: fr3_link1
4: fr3_joint2
5: fr3_link2
6: fr3_joint3
7: fr3_link3
8: fr3_joint4
9: fr3_link4
10: fr3_joint5
11: fr3_link5
12: fr3_joint6
13: fr3_link6
14: fr3_joint7
15: fr3_link7
16: fr3_joint8
17: fr3_link8
18: fr3_hand_joint
19: fr3_hand
20: fr3_finger_joint1
21: fr3_leftfinger
22: fr3_finger_joint2
23: fr3_rightfinger
24: fr3_hand_tcp_joint
25: fr3_hand_tcp
starting the controller


ValueError: The index of the Frame is outside the bounds.